# Datathon Cajamar UniversityHack 2022 - Cajamar Water Footprint

## Script Predicción - Equipo DPNA

- Pablo Torrijos Arenas
- Mykola Mandzyak Melnyk
- Alejandro Gómez Escribano


## Imports y leer csv

In [8]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

import xgboost as xgb

from category_encoders import TargetEncoder

pd.options.display.max_info_rows = 30000000

sns.set_style('darkgrid')

# Tamaño figuras
plt.rcParams["figure.figsize"] = (15,6)

# Extracción, transformación y carga de los datos

Vamos a leer la base de datos proporcionada:

In [2]:
df = pd.read_csv('data/Modelar_UH2022.txt', sep='|', index_col='ID', parse_dates=True)
df['SAMPLETIME'] = pd.to_datetime(df.SAMPLETIME)
df = df.sort_values(["ID", 'SAMPLETIME'])
df.head()

,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH
ID,,,,,
0,2019-02-01 00:39:36,331710,0.0,0,0.0
0,2019-02-01 01:39:36,331710,0.0,0,0.0
0,2019-02-01 02:39:35,331710,0.0,0,0.0
0,2019-02-01 03:39:35,331710,0.0,0,0.0
0,2019-02-01 04:39:35,331710,0.0,0,0.0


## Valores negativos y NaN

Primero vamos a cambiar de signo los valores negativos:

In [3]:
df['READINGINTEGER'] = df['READINGINTEGER'].apply(lambda x : x if x >= 0 else -x)
df['READINGTHOUSANDTH'] = df['READINGTHOUSANDTH'].apply(lambda x : x if x >= 0 else -x)
df['DELTAINTEGER'] = df['DELTAINTEGER'].apply(lambda x : x if x >= 0 else -x)
df['DELTATHOUSANDTH'] = df['DELTATHOUSANDTH'].apply(lambda x : x if x >= 0 else -x)

Y después establecemos a 0 los valores `NaN` (principalmente en las columnas `READINGTHOUSANDTH` y `DELTATHOUSANDTH`, que posteriormente no vamos a utilizar):

In [4]:
df.fillna(0, inplace=True)
df.loc[292].sort_values('SAMPLETIME').head(5)

,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH
ID,,,,,
292,2019-02-01 00:00:00,17433250,0.0,320,0.0
292,2019-02-01 00:15:00,17433560,0.0,310,0.0
292,2019-02-01 00:30:00,17433880,0.0,320,0.0
292,2019-02-01 00:45:00,17434190,0.0,310,0.0
292,2019-02-01 01:00:00,17434820,0.0,630,0.0


## Contadores con horas distintas

Vamos a cambiar los minutos y segundos por 0's:

In [5]:
df['SAMPLETIME'] = df['SAMPLETIME'].apply(lambda x:x.replace(minute=0,second=0))
df

,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH
ID,,,,,
0,2019-02-01 00:00:00,331710,0.0,0,0.0
0,2019-02-01 01:00:00,331710,0.0,0,0.0
0,2019-02-01 02:00:00,331710,0.0,0,0.0
0,2019-02-01 03:00:00,331710,0.0,0,0.0
0,2019-02-01 04:00:00,331710,0.0,0,0.0
...,...,...,...,...,...
2749,2019-10-11 07:00:00,164507,0.0,2,0.0
2756,2019-04-06 04:00:00,349758,0.0,0,0.0
2756,2019-04-06 05:00:00,349758,0.0,0,0.0


In [6]:
df_temp = df.groupby(['ID','SAMPLETIME']).size().sort_values()
unique2, counts2 = np.unique(df_temp, return_counts=True)
dict(zip(unique2, counts2))

{1: 21264774, 3: 2, 4: 34882, 5: 104}

Empezamos con el caso más fácil, cuando hay 4 horas. Aquí, tenemos las horas :00, :15, :30 y :45. También haremos el mismo proceso cuando contamos con 3 horas (solo en 2 casos), ya que es un caso similar:

In [7]:
df_borrar = df_temp[(df_temp == 4) | (df_temp == 3)].sort_index().to_frame()
df_borrar

0
ID   SAMPLETIME            
248  2019-02-01 00:00:00  4
     2019-02-01 01:00:00  4
     2019-02-01 02:00:00  4
     2019-02-01 03:00:00  4
     2019-02-01 04:00:00  4
...                      ..
1416 2020-01-31 19:00:00  4
     2020-01-31 20:00:00  4
     2020-01-31 21:00:00  4
     2020-01-31 22:00:00  4
     2020-01-31 23:00:00  4

[34884 rows x 1 columns]

In [8]:
for indx in tqdm(df_borrar.index):
    df2 = df.loc[indx[0]]
    filas = df2.loc[(df2['SAMPLETIME'] >= indx[1]) & 
                    (df2['SAMPLETIME'] < (indx[1] + pd.Timedelta(hours=1)))].sort_values('SAMPLETIME')
    df2.loc[df2['SAMPLETIME'] == indx[1],'DELTAINTEGER'] = filas['DELTAINTEGER'].sum()
    df2.loc[df2['SAMPLETIME'] == indx[1],'READINGINTEGER'] = filas['READINGINTEGER'].max()

  0%|          | 0/34884 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
100%|██████████| 34884/34884 [01:28<00:00, 393.20it/s]


Ahora seguimos con las horas que tienen 5 ocurrencias. Este caso es más complicado, ya que sus minutos son :00, :14, :29, :44 y :59. Este último se debería juntar con la siguiente hora.

In [9]:
df_borrar = df_temp[df_temp == 5].sort_index().to_frame()
df_borrar

0
ID   SAMPLETIME            
248  2019-03-04 05:00:00  5
     2019-04-17 06:00:00  5
     2019-04-18 06:00:00  5
     2019-04-21 06:00:00  5
     2019-04-27 06:00:00  5
...                      ..
1416 2019-12-05 05:00:00  5
     2019-12-07 05:00:00  5
     2019-12-11 05:00:00  5
     2019-12-12 05:00:00  5
     2020-01-16 05:00:00  5

[104 rows x 1 columns]

In [10]:
for indx in tqdm(df_borrar.index):
    df2 = df.loc[indx[0]]
    filas = df2.loc[(df2['SAMPLETIME'] >= indx[1]) & 
                    (df2['SAMPLETIME'] < (indx[1] + pd.Timedelta(hours=1)))].sort_values('SAMPLETIME')
    
    max_actual = filas.iloc[filas['SAMPLETIME'].argmax()]['DELTAINTEGER']
    siguiente = df2.loc[df2['SAMPLETIME'] == indx[1] + pd.Timedelta(hours=1)]['DELTAINTEGER'].values[0]
    
    # Añadimos el delta del valor con máximo READINGINTEGER a la siguiente hora
    df2.loc[df2['SAMPLETIME'] == indx[1] + pd.Timedelta(hours=1),'DELTAINTEGER'] = max_actual + siguiente
    
    # Establecemos el delta de la fila que hemos cogido a 0 y su READINGINTEGER al minimo de esa hora para que no se tenga en cuenta
    df2.loc[df2['READINGINTEGER'] == filas['READINGINTEGER'].max(),'DELTAINTEGER'] = 0
    df2.loc[df2['READINGINTEGER'] == filas['READINGINTEGER'].max(),'READINGINTEGER'] = filas['READINGINTEGER'].min()
    
    df2.loc[df2['SAMPLETIME'] == indx[1],'DELTAINTEGER'] = (df2.loc[df2['SAMPLETIME'] == indx[1],'DELTAINTEGER'].sum())
    df2.loc[df2['SAMPLETIME'] == indx[1],'READINGINTEGER'] = (df2.loc[df2['SAMPLETIME'] == indx[1],'READINGINTEGER'].max())

  0%|          | 0/104 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
100%|██████████| 104/104 [00:00<00:00, 165.11it/s]


Y por último, como hemos dejado las horas que coincidían repetidas con los mismos valores, vamos a eliminarlas para quedaros solo con una:

In [11]:
df = df.reset_index().drop_duplicates()
df = df.set_index(['ID'])
df.index.drop_duplicates()

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2739, 2742, 2743, 2744, 2745, 2746, 2747, 2748, 2749, 2756],
           dtype='int64', name='ID', length=2747)

Podemos ver cómo ahora tenemos el número de horas que queríamos:

In [12]:
df.groupby(['SAMPLETIME']).size().count()

8759

Y por último vamos a guardar los resultados en un archivo intermedio:

In [ ]:
df.to_csv('data/Modelar_Procesado.csv')

## Corregir contadores decrecientes

Para corregir los contadores decrecientes, vamos ralizamos el siguiente proceso:

Para cada `ID`, si el valor de `READINGINTEGER` ha decrecido de una hora a la inmediatamente posterior, sumamos a todos los valores posteriores a los de esa hora el doble de la diferencia de dichas horas:

In [13]:
# Para cada contador (índice)
for i in tqdm(df.index.drop_duplicates()):
    df_temp = df.loc[i]
        
    # Comprobamos uno por uno todos los valores comparándolos con el anterior
    for j in range(1,len(df_temp)):
        if df_temp.iloc[j]["READINGINTEGER"] < df_temp.iloc[j-1]["READINGINTEGER"]:
            
            new = 2 * (df_temp.iloc[j-1]["READINGINTEGER"] - df_temp.iloc[j]["READINGINTEGER"])
            
            df.loc[i].loc[df_temp['SAMPLETIME'] >= df_temp.iloc[j]["SAMPLETIME"],'READINGINTEGER'] = df_temp.loc[df_temp['SAMPLETIME'] >= df_temp.iloc[j]["SAMPLETIME"],'READINGINTEGER'] + new

  0%|          | 3/2747 [00:05<1:15:51,  1.66s/it]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
100%|██████████| 2747/2747 [1:05:27<00:00,  1.43s/it]


Y guardamos un DataFrame temporal:

In [14]:
df.to_csv('data/Modelar_Procesado_2.csv')

## Corregir cambios drásticos desde una hora a la siguiente

El proceso para eliminar esos "escalones" de los ID's, ya sean por que los hemos introducido al eliminar los valores decrecientes o porque ya estuvieran en las bases de datos, será el siguiente:

Para cada ID, creamos dos columnas nuevas (`DELTA_I` y `DELTA_H`) con la diferencia en `READINGINTEGER` y `SAMPLETIME` entre cada fila y la anterior. Así, cuando haya pasado solo una hora entre una entrada y otra (no haya valores perdidos), si la diferencia de `READINGINTEGER` es mayor que el máximo `READINGINTEGER` menos el mínimo dividido por 200 (valor que parece razonable, si el consumo fuera uniforme sería 8759), restamos a todas las filas posteriores esa diferencia. Además, esto lo hacemos solo si el consumo en esa hora es mayor que 200 (lo establecemos como valor mínimo porque es, por ejemplo, lo que puede gastar una persona duchándose).

In [15]:
pd.set_option('mode.chained_assignment', None)

plt.rcParams["figure.figsize"] = (15,8)

MENOS_FILTRO = [418, 516, 537, 741, 805, 1136, 1696]

# Para cada contador (índice)
for i in tqdm(df.index.drop_duplicates()):
    df_temp = df.loc[i]
        
    df_temp['DELTA_I'] = df_temp.diff()['READINGINTEGER']
    df_temp['DELTA_H'] = df_temp.diff()['SAMPLETIME']
        
    diff = df_temp.READINGINTEGER.max() - df_temp.READINGINTEGER.min()

    maxDelta = diff / 200  # 200 para 8759 horas
    
    if i in MENOS_FILTRO:
        maxDelta = maxDelta * 2

    # Filtramos si cambia de una hora para otra más de lo que "debería"
    df_temp_2 = df_temp.loc[(df_temp['DELTA_I'] > maxDelta) & (df_temp['DELTA_H'] == '0 days 01:00:00')]
        
    if (df_temp_2.count()["SAMPLETIME"] != 0) and (maxDelta > 200):

        for j in range(0,len(df_temp_2)):
            filtro = df_temp['SAMPLETIME'] >= df_temp_2.iloc[j]["SAMPLETIME"]

            # A todos los valores posteriores al salto, les restamos la diferencia de READINGINTEGER entre j y j-1 (que siempre > 0)
            df_temp.loc[filtro,'READINGINTEGER'] = df_temp.loc[filtro,'READINGINTEGER'] - df_temp_2.iloc[j]["DELTA_I"]
                    
        df.loc[i] = df_temp

plt.rcParams["figure.figsize"] = (15,6)

100%|██████████| 2747/2747 [03:59<00:00, 11.49it/s]


In [16]:
df.to_csv('data/Modelar_Procesado_3.csv')

## Añadir datos de festividades

Primero vamos a crear una columna específica para especificar el año, mes, día, hora y día de la semana:

In [17]:
df['YEAR'] = df['SAMPLETIME'].dt.strftime('%Y').astype(int)
df['MONTH'] = df['SAMPLETIME'].dt.strftime('%m').astype(int)
df['DAY'] = df['SAMPLETIME'].dt.strftime('%e').astype(int)
df['HOUR'] = df['SAMPLETIME'].dt.strftime('%H').astype(int)
df['DAYWEEK'] = df['SAMPLETIME'].dt.strftime('%u').astype(int)

Y a continuación, vamos a añadir datos de festividades tanto nacionales, como autonómicas (de la Comunidad Valenciana) como locales (de Valencia):

In [18]:
#FESTIVIDAD NACIONAL
anos = [2019,2020]

count_n = [[1,1,1,1,1,1,2],[2,1,1,1,1,2]]
meses_n = [[1,4,5,8,10,11,12],[1,4,5,8,10,12]]
nacionales = [[1,19,1,15,12,1,6,25],[1,6,10,1,15,12,8,25]]

count_a = [[2,1,1],[1,2,1,1]]
meses_a = [[4,6,10],[3,4,6,10]]
autonomicos = [[22,25,24,9],[19,13,25,24,9]]

count_l = [[1],[1]]
meses_l = [[1],[1]]
locales = [[22],[22]]

df['NACIONAL'] = 0
df['AUTONOMICO'] = 0
df['LOCAL'] = 0

nombres = ["NACIONAL", "AUTONOMICO", "LOCAL"]
aux_c = [count_n, count_a, count_l]
aux_m = [meses_n, meses_a, meses_l]
aux_d = [nacionales, autonomicos, locales]

for colum in nombres:
    meses = aux_m.pop(0)
    count = aux_c.pop(0)
    days = aux_d.pop(0)
    for ano in anos:
        meses2 = meses.pop(0)
        count2 = count.pop(0)
        days2 = days.pop(0)
        for m in meses2:
            cuenta = count2.pop(0)
            for d in range(0, cuenta): 
                df.loc[(df["YEAR"]==ano) & (df["MONTH"]==m) & (df["DAY"]==days2.pop(0)), colum] = 1
df

,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL
ID,,,,,,,,,,,,,
0,2019-02-01 00:00:00,331710.0,0.0,0,0.0,2019,2,1,0,5,0,0,0
0,2019-02-01 01:00:00,331710.0,0.0,0,0.0,2019,2,1,1,5,0,0,0
0,2019-02-01 02:00:00,331710.0,0.0,0,0.0,2019,2,1,2,5,0,0,0
0,2019-02-01 03:00:00,331710.0,0.0,0,0.0,2019,2,1,3,5,0,0,0
0,2019-02-01 04:00:00,331710.0,0.0,0,0.0,2019,2,1,4,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,2019-10-11 07:00:00,164507.0,0.0,2,0.0,2019,10,11,7,5,0,0,0
2756,2019-04-06 04:00:00,349758.0,0.0,0,0.0,2019,4,6,4,6,0,0,0
2756,2019-04-06 05:00:00,349758.0,0.0,0,0.0,2019,4,6,5,6,0,0,0


Y guardamos el archivo temporal:

In [ ]:
df.to_csv('data/Modelar_Procesado_4.csv')

## Unir DataFrame con el de Precipitaciones, Temperatura y Humedad

Vamos a enriquecer nuestros datos con la información meteorológica por horas en varias estaciones meteorológicas de la ciudad de Valencia. Si leemos la base de datos, obtenemos lo siguiente:

In [3]:
df2 = pd.read_csv('data/PrecHumTempValencia.csv', sep='\;', encoding='ISO-8859-1')
df2.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,"""Fecha Hora",Estación,Temp,H.Rel,R.Sol,"Precip"""
0,"""01/01/2020 0:00",Avda. Francia,"6,6",890.0,0.0,"0"""
1,"""01/01/2020 1:00",Avda. Francia,"6,2",890.0,0.0,"0"""
2,"""01/01/2020 2:00",Avda. Francia,"5,9",900.0,0.0,"0"""
3,"""01/01/2020 3:00",Avda. Francia,"5,5",900.0,0.0,"0"""
4,"""01/01/2020 4:00",Avda. Francia,"5,3",890.0,0.0,"0"""


Quitamos las comillas, convertimos los valores de string a números y establecemos la Fecha como índice:

In [4]:
df2 = df2.replace('"','', regex=True)
df2 = df2.replace(',','.', regex=True)
df2 = df2.rename(columns={"\"Fecha Hora": "Fecha", "Precip\"": "Precip"})
df2["Temp"] = pd.to_numeric(df2["Temp"])
df2["H.Rel"] = pd.to_numeric(df2["H.Rel"])
df2["R.Sol"] = pd.to_numeric(df2["R.Sol"])
df2["Precip"] = pd.to_numeric(df2["Precip"])
df2['Fecha'] = pd.to_datetime(df2.Fecha)
df2 = df2.set_index("Fecha")
df2

,Estación,Temp,H.Rel,R.Sol,Precip
Fecha,,,,,
2020-01-01 00:00:00,Avda. Francia,6.6,890.0,0.0,0.0
2020-01-01 01:00:00,Avda. Francia,6.2,890.0,0.0,0.0
2020-01-01 02:00:00,Avda. Francia,5.9,900.0,0.0,0.0
2020-01-01 03:00:00,Avda. Francia,5.5,900.0,0.0,0.0
2020-01-01 04:00:00,Avda. Francia,5.3,890.0,0.0,0.0
...,...,...,...,...,...
2018-12-23 01:00:00,Politécnico,11.5,760.0,20.0,0.0
2018-12-23 02:00:00,Politécnico,11.0,770.0,10.0,0.0
2018-12-23 03:00:00,Politécnico,10.9,770.0,10.0,0.0


Una vez hecho esto, agrupamos por Fecha para combinar los valores de las distintas estaciones con la media:

In [5]:
df2 = df2.sort_index()
df_group = df2.groupby("Fecha").mean()
df_group

,Temp,H.Rel,R.Sol,Precip
Fecha,,,,
2018-01-01 00:00:00,14.20,380.0,0.0,0.0
2018-01-01 01:00:00,12.80,460.0,0.0,0.0
2018-01-01 02:00:00,12.10,460.0,0.0,0.0
2018-01-01 03:00:00,11.20,450.0,0.0,0.0
2018-01-01 04:00:00,10.60,460.0,0.0,0.0
...,...,...,...,...
2020-12-31 19:00:00,11.15,442.0,4.0,0.0
2020-12-31 20:00:00,10.51,474.0,14.0,0.0
2020-12-31 21:00:00,10.37,506.0,4.0,0.0


Pero podemos ver cómo hay valores perdidos:

In [6]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26304 entries, 2018-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Temp    25882 non-null  float64
 1   H.Rel   25170 non-null  float64
 2   R.Sol   25850 non-null  float64
 3   Precip  25858 non-null  float64
dtypes: float64(4)
memory usage: 1.0 MB


In [7]:
pd.date_range(start=pd.Timestamp('2018-01-01T00'), end=pd.Timestamp('2020-12-31T23'), freq='60T')

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00', '2018-01-01 03:00:00',
               '2018-01-01 04:00:00', '2018-01-01 05:00:00',
               '2018-01-01 06:00:00', '2018-01-01 07:00:00',
               '2018-01-01 08:00:00', '2018-01-01 09:00:00',
               ...
               '2020-12-31 14:00:00', '2020-12-31 15:00:00',
               '2020-12-31 16:00:00', '2020-12-31 17:00:00',
               '2020-12-31 18:00:00', '2020-12-31 19:00:00',
               '2020-12-31 20:00:00', '2020-12-31 21:00:00',
               '2020-12-31 22:00:00', '2020-12-31 23:00:00'],
              dtype='datetime64[ns]', length=26304, freq='60T')

Como no son demasiados valores perdidos y no son variables críticas, creemos que imputándolos con un `SimpleImputer` podemos obtener valores adecuados:

In [8]:
imp = SimpleImputer(missing_values=np.nan)
idf = pd.DataFrame(imp.fit_transform(df_group))
idf.columns = df_group.columns
idf.index = df_group.index
idf

,Temp,H.Rel,R.Sol,Precip
Fecha,,,,
2018-01-01 00:00:00,14.20,380.0,0.0,0.0
2018-01-01 01:00:00,12.80,460.0,0.0,0.0
2018-01-01 02:00:00,12.10,460.0,0.0,0.0
2018-01-01 03:00:00,11.20,450.0,0.0,0.0
2018-01-01 04:00:00,10.60,460.0,0.0,0.0
...,...,...,...,...
2020-12-31 19:00:00,11.15,442.0,4.0,0.0
2020-12-31 20:00:00,10.51,474.0,14.0,0.0
2020-12-31 21:00:00,10.37,506.0,4.0,0.0


Y una vez los tenemos, los agrupamos a la tabla original:

In [9]:
dfjoin = df.set_index('SAMPLETIME', append=True).reset_index(level=0)
dfjoin = dfjoin.join(idf)
dfjoin = dfjoin.set_index('ID', append=True).reset_index(level=0)
dfjoin = dfjoin.rename(columns={"level_0": "SAMPLETIME"})
dfjoin

,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip
ID,,,,,,,,,,,,,,,,,
0,2019-02-01 00:00:00,331710.0,0.0,0,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0
1,2019-02-01 00:00:00,5825.0,0.0,0,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0
2,2019-02-01 00:00:00,7951.0,0.0,0,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0
3,2019-02-01 00:00:00,150316.0,0.0,37,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0
4,2019-02-01 00:00:00,113385.0,0.0,14,56.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,2020-01-31 23:00:00,11813.0,0.0,0,0.0,2020,1,31,23,5,0,0,0,14.89,617.0,0.0,0.0
2735,2020-01-31 23:00:00,63785.0,0.0,0,0.0,2020,1,31,23,5,0,0,0,14.89,617.0,0.0,0.0
2736,2020-01-31 23:00:00,32783.0,0.0,36,0.0,2020,1,31,23,5,0,0,0,14.89,617.0,0.0,0.0


## Unir DataFrame con el de Precios

También hemos conseguido un DataSet público de ocupación y tarifas diarias del turismo de Valencia, que uniremos asímismo a nuestro dataset

In [10]:
dfjoin['FECHA'] = dfjoin['SAMPLETIME'].dt.strftime('%F')
dfjoin['FECHA'] = pd.to_datetime(dfjoin.FECHA)
dfjoin

,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,FECHA
ID,,,,,,,,,,,,,,,,,,
0,2019-02-01 00:00:00,331710.0,0.0,0,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0,2019-02-01
1,2019-02-01 00:00:00,5825.0,0.0,0,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0,2019-02-01
2,2019-02-01 00:00:00,7951.0,0.0,0,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0,2019-02-01
3,2019-02-01 00:00:00,150316.0,0.0,37,0.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0,2019-02-01
4,2019-02-01 00:00:00,113385.0,0.0,14,56.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0,2019-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,2020-01-31 23:00:00,11813.0,0.0,0,0.0,2020,1,31,23,5,0,0,0,14.89,617.0,0.0,0.0,2020-01-31
2735,2020-01-31 23:00:00,63785.0,0.0,0,0.0,2020,1,31,23,5,0,0,0,14.89,617.0,0.0,0.0,2020-01-31
2736,2020-01-31 23:00:00,32783.0,0.0,36,0.0,2020,1,31,23,5,0,0,0,14.89,617.0,0.0,0.0,2020-01-31


In [11]:
precios_df = pd.read_excel('data\DATOS DIAS.xlsx')
precios_df = precios_df.astype({'FECHA': 'datetime64[ns]'})
precios_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   FECHA                        425 non-null    datetime64[ns]
 1   TARIFA DIARIA COMPLETA       425 non-null    object        
 2   OCUPACIÓN DIARIA COMPLETA    425 non-null    float64       
 3   TARIFA DIARIA COMPARTIDA     425 non-null    float64       
 4   OCUPACIÓN DIARIA COMPARTIDA  425 non-null    float64       
 5   TARIFA DIARIA PRIVADA        425 non-null    float64       
 6   OCUPACIÓN DIARIA PRIVADA     425 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 23.4+ KB


Obteniendo el siguiente resultado:

In [12]:
precios_df = precios_df.set_index('FECHA')
df3 = dfjoin.set_index('FECHA', append=True).reset_index(level=0)
df3 = df3.join(precios_df)
df3 = df3.set_index('ID', append=True).reset_index(level=0)
dfjoin = df3.rename(columns={"level_0": "FECHA"})
dfjoin

,FECHA,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,HOUR,...,Temp,H.Rel,R.Sol,Precip,TARIFA DIARIA COMPLETA,OCUPACIÓN DIARIA COMPLETA,TARIFA DIARIA COMPARTIDA,OCUPACIÓN DIARIA COMPARTIDA,TARIFA DIARIA PRIVADA,OCUPACIÓN DIARIA PRIVADA
ID,,,,,,,,,,,,,,,,,,,,,
0,2019-02-01,2019-02-01 00:00:00,331710.0,0.0,0,0.0,2019,2,1,0,...,9.10,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
1,2019-02-01,2019-02-01 00:00:00,5825.0,0.0,0,0.0,2019,2,1,0,...,9.10,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
2,2019-02-01,2019-02-01 00:00:00,7951.0,0.0,0,0.0,2019,2,1,0,...,9.10,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
3,2019-02-01,2019-02-01 00:00:00,150316.0,0.0,37,0.0,2019,2,1,0,...,9.10,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
4,2019-02-01,2019-02-01 00:00:00,113385.0,0.0,14,56.0,2019,2,1,0,...,9.10,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,2020-01-31,2020-01-31 23:00:00,11813.0,0.0,0,0.0,2020,1,31,23,...,14.89,617.0,0.0,0.0,71.25,0.536,44.00,0.165,29.45,0.351
2735,2020-01-31,2020-01-31 23:00:00,63785.0,0.0,0,0.0,2020,1,31,23,...,14.89,617.0,0.0,0.0,71.25,0.536,44.00,0.165,29.45,0.351
2736,2020-01-31,2020-01-31 23:00:00,32783.0,0.0,36,0.0,2020,1,31,23,...,14.89,617.0,0.0,0.0,71.25,0.536,44.00,0.165,29.45,0.351


## Unir DataFrame con el de Hoteles

Por último, también contamos con otro DataSet de ocupación de Hoteles (en este caso con frecuencia mensual). 

In [ ]:
hoteles_df = pd.read_excel('data\VIAJEROS Y PERNOTACIONES DE HOTELES Y APARTAMENTOS.xlsx')
hoteles_df = hoteles_df.drop(hoteles_df.columns[[0,1,2,3,4,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25]], axis=1)

In [14]:
dfjoin['FECHA2'] = dfjoin['SAMPLETIME'].dt.strftime('%Y-%m')
dfjoin['FECHA2'] = pd.to_datetime(dfjoin.FECHA2)
dfjoin

,FECHA,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,HOUR,...,H.Rel,R.Sol,Precip,TARIFA DIARIA COMPLETA,OCUPACIÓN DIARIA COMPLETA,TARIFA DIARIA COMPARTIDA,OCUPACIÓN DIARIA COMPARTIDA,TARIFA DIARIA PRIVADA,OCUPACIÓN DIARIA PRIVADA,FECHA2
ID,,,,,,,,,,,,,,,,,,,,,
0,2019-02-01,2019-02-01 00:00:00,331710.0,0.0,0,0.0,2019,2,1,0,...,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378,2019-02-01
1,2019-02-01,2019-02-01 00:00:00,5825.0,0.0,0,0.0,2019,2,1,0,...,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378,2019-02-01
2,2019-02-01,2019-02-01 00:00:00,7951.0,0.0,0,0.0,2019,2,1,0,...,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378,2019-02-01
3,2019-02-01,2019-02-01 00:00:00,150316.0,0.0,37,0.0,2019,2,1,0,...,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378,2019-02-01
4,2019-02-01,2019-02-01 00:00:00,113385.0,0.0,14,56.0,2019,2,1,0,...,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378,2019-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,2020-01-31,2020-01-31 23:00:00,11813.0,0.0,0,0.0,2020,1,31,23,...,617.0,0.0,0.0,71.25,0.536,44.00,0.165,29.45,0.351,2020-01-01
2735,2020-01-31,2020-01-31 23:00:00,63785.0,0.0,0,0.0,2020,1,31,23,...,617.0,0.0,0.0,71.25,0.536,44.00,0.165,29.45,0.351,2020-01-01
2736,2020-01-31,2020-01-31 23:00:00,32783.0,0.0,36,0.0,2020,1,31,23,...,617.0,0.0,0.0,71.25,0.536,44.00,0.165,29.45,0.351,2020-01-01


Después de realizar algunos cambios en el mismo, obtenemos lo siguiente:

In [15]:
hoteles_df['AÑO'] = hoteles_df['AÑO'].astype(str)
hoteles_df['MES'] = hoteles_df['MES'].astype(str)
hoteles_df['FECHA2'] = hoteles_df['AÑO'] + "-" + hoteles_df['MES']
hoteles_df['FECHA2'] = pd.to_datetime(hoteles_df.FECHA2)
hoteles_df = hoteles_df.set_index('FECHA2')
hoteles_df

,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL,MES,AÑO
FECHA2,,,,
2019-01-01,303245,40.955,1,2019
2019-02-01,311599,46.870,2,2019
2019-03-01,419853,53.790,3,2019
2019-04-01,477711,64.705,4,2019
2019-05-01,490101,65.195,5,2019
2019-06-01,496639,67.390,6,2019
2019-07-01,555251,72.695,7,2019
2019-08-01,610223,74.975,8,2019
2019-09-01,482365,69.155,9,2019


Y podemos unirlo a nuestro DataSet principal:

In [16]:
df3 = dfjoin.set_index('FECHA2', append=True).reset_index(level=0)
df3 = df3.join(hoteles_df)
df3 = df3.set_index('ID', append=True).reset_index(level=0)
dfjoin = df3.rename(columns={"level_0": "FECHA2"})
dfjoin

,FECHA2,FECHA,SAMPLETIME,READINGINTEGER,READINGTHOUSANDTH,DELTAINTEGER,DELTATHOUSANDTH,YEAR,MONTH,DAY,...,TARIFA DIARIA COMPLETA,OCUPACIÓN DIARIA COMPLETA,TARIFA DIARIA COMPARTIDA,OCUPACIÓN DIARIA COMPARTIDA,TARIFA DIARIA PRIVADA,OCUPACIÓN DIARIA PRIVADA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL,MES,AÑO
ID,,,,,,,,,,,,,,,,,,,,,
0,2019-02-01,2019-02-01,2019-02-01 00:00:00,331710.0,0.0,0,0.0,2019,2,1,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
1,2019-02-01,2019-02-01,2019-02-01 00:00:00,5825.0,0.0,0,0.0,2019,2,1,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
2,2019-02-01,2019-02-01,2019-02-01 00:00:00,7951.0,0.0,0,0.0,2019,2,1,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
3,2019-02-01,2019-02-01,2019-02-01 00:00:00,150316.0,0.0,37,0.0,2019,2,1,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
4,2019-02-01,2019-02-01,2019-02-01 00:00:00,113385.0,0.0,14,56.0,2019,2,1,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,2020-01-01,2020-01-31,2020-01-31 23:00:00,11813.0,0.0,0,0.0,2020,1,31,...,71.25,0.536,44.00,0.165,29.45,0.351,339021,43.13,1,2020
2735,2020-01-01,2020-01-31,2020-01-31 23:00:00,63785.0,0.0,0,0.0,2020,1,31,...,71.25,0.536,44.00,0.165,29.45,0.351,339021,43.13,1,2020
2736,2020-01-01,2020-01-31,2020-01-31 23:00:00,32783.0,0.0,36,0.0,2020,1,31,...,71.25,0.536,44.00,0.165,29.45,0.351,339021,43.13,1,2020


Por último borramos variables que no queremos:

In [ ]:
dfjoin = dfjoin.drop(["FECHA2", "FECHA", "MES", "AÑO"], axis=1)

Y gardamos el dataset "final" (a falta de imputar los valores perdidos):

In [17]:
dfjoin.to_csv('data/Modelar_Procesado_DEF.csv')

## Imputación de valores

Primero vamos a borrar las columnas `READINGTHOUSANDTH`, `DELTAINTEGER` Y `DELTATHOUSANDTH`, ya que vamos a predecir los valores perdidos. Además, también borramos los `MINUTE` y `SECONDS`. Además, borramos varias columnas de ocupación y tarifas ya que los datos que obtenemos de ellas son prácticamente los mismos:

In [7]:
df = df.drop(['FECHA2', 'FECHA', 'READINGTHOUSANDTH', 'DELTAINTEGER', 'DELTATHOUSANDTH'], axis=1)
df = df.drop(['TARIFA DIARIA COMPLETA', 'TARIFA DIARIA COMPARTIDA', 'OCUPACIÓN DIARIA COMPARTIDA', 'TARIFA DIARIA PRIVADA', 'OCUPACIÓN DIARIA PRIVADA'], axis=1)

### Rellenar valores perdidos con imputador de Pandas

Para los casos en los que faltan valores intermedios de `READINGINTEGER` entre dos horas, simplemente vamos a trazar una línea recta entre ambos (interpolamos). Esto lo conseguimos directamente con la función `.interpolate()` Pandas.

En el resto de casos, extrapolamos con un `IterativeImputer` que utiliza el resto de variables que hemos añadido.

Para ello, primero tenemos que comprobar qué fechas faltan, y añadirles el resto de columnas que hemos creado. Como hemos comprobado que el ID 0 cuenta con todos los valores posibles de horas (los 8759), vamos a utilizar sus columnas:

In [8]:
time_values = df.loc[0]["SAMPLETIME"].values
time_values

array(['2019-02-01T00:00:00.000000000', '2019-02-01T01:00:00.000000000',
       '2019-02-01T02:00:00.000000000', ...,
       '2020-01-31T21:00:00.000000000', '2020-01-31T22:00:00.000000000',
       '2020-01-31T23:00:00.000000000'], dtype='datetime64[ns]')

In [9]:
valores_base = df.loc[0].set_index('SAMPLETIME')
valores_base

,READINGINTEGER,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL,MES,AÑO
SAMPLETIME,,,,,,,,,,,,,,,,,,
2019-02-01 00:00:00,331710.0,2019,2,1,0,5,0,0,0,9.10,700.0,0.0,0.0,0.561,311599,46.87,2,2019
2019-02-01 01:00:00,331710.0,2019,2,1,1,5,0,0,0,8.90,670.0,0.0,0.0,0.561,311599,46.87,2,2019
2019-02-01 02:00:00,331710.0,2019,2,1,2,5,0,0,0,8.80,640.0,0.0,0.0,0.561,311599,46.87,2,2019
2019-02-01 03:00:00,331710.0,2019,2,1,3,5,0,0,0,8.30,660.0,0.0,0.0,0.561,311599,46.87,2,2019
2019-02-01 04:00:00,331710.0,2019,2,1,4,5,0,0,0,9.70,550.0,0.0,0.0,0.561,311599,46.87,2,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31 19:00:00,434941.0,2020,1,31,19,5,0,0,0,16.77,591.0,0.0,0.0,0.536,339021,43.13,1,2020
2020-01-31 20:00:00,434951.0,2020,1,31,20,5,0,0,0,16.13,594.0,0.0,0.0,0.536,339021,43.13,1,2020
2020-01-31 21:00:00,434952.0,2020,1,31,21,5,0,0,0,15.57,612.0,0.0,0.0,0.536,339021,43.13,1,2020


Una vez tenemos esto, creamos las filas que faltaban, añadiendo un `NaN` como `SAMPLETIME`:

In [10]:
dic = {'ID': [], 'SAMPLETIME': [], 'READINGINTEGER': [], 'YEAR': [], 'MONTH': [], 'DAY': [], 
       'HOUR': [], 'DAYWEEK': [], 'NACIONAL': [], 'AUTONOMICO': [], 'LOCAL': [], 
       'Temp': [], 'H.Rel': [], 'R.Sol': [], 'Precip': [], 
       'OCUPACIÓN DIARIA COMPLETA': [], 'PERNOTACIONES TOTAL': [], 'OCUPACIÓN TOTAL': []}
list_i = []
						
# Para cada índice
for i in tqdm(df.index.drop_duplicates()):
    df_temp = df.loc[i]
    
    # Si le faltan valores
    if df_temp.size != df.loc[0].size:
        time_values_temp = df_temp["SAMPLETIME"].values
        list_i.append(i)
        
        # Para cada valor posible de fecha/hora
        for time in time_values:
            
            # Si no existe para el ID actual, lo añadimos
            if time not in time_values_temp:
                fila = valores_base.loc[time]
                dic['ID'].append(i)
                dic['SAMPLETIME'].append(time)
                dic['READINGINTEGER'].append(np.nan)
                dic['YEAR'].append(fila['YEAR'])
                dic['MONTH'].append(fila['MONTH'])
                dic['DAY'].append(fila['DAY'])
                dic['HOUR'].append(fila['HOUR'])
                dic['DAYWEEK'].append(fila['DAYWEEK'])
                dic['NACIONAL'].append(fila['NACIONAL'])
                dic['AUTONOMICO'].append(fila['AUTONOMICO'])
                dic['LOCAL'].append(fila['LOCAL'])
                dic['Temp'].append(fila['Temp'])
                dic['H.Rel'].append(fila['H.Rel'])
                dic['R.Sol'].append(fila['R.Sol'])
                dic['Precip'].append(fila['Precip'])
                dic['OCUPACIÓN DIARIA COMPLETA'].append(fila['OCUPACIÓN DIARIA COMPLETA'])
                dic['PERNOTACIONES TOTAL'].append(fila['PERNOTACIONES TOTAL'])
                dic['OCUPACIÓN TOTAL'].append(fila['OCUPACIÓN TOTAL'])

100%|██████████| 2747/2747 [08:33<00:00,  5.35it/s]


In [11]:
df_dictionary = pd.DataFrame(dic)
df_dictionary = df_dictionary.set_index('ID')
df = pd.concat([df, df_dictionary], ignore_index=False)
df = df.sort_values(["ID", 'SAMPLETIME'])
df = df.drop(['MES','AÑO'],axis=1)

In [12]:
df

,SAMPLETIME,READINGINTEGER,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL
ID,,,,,,,,,,,,,,,,,
0,2019-02-01 00:00:00,331710.0,2019.0,2.0,1.0,0.0,5.0,0.0,0.0,0.0,9.10,700.0,0.0,0.0,0.561,311599.0,46.87
0,2019-02-01 01:00:00,331710.0,2019.0,2.0,1.0,1.0,5.0,0.0,0.0,0.0,8.90,670.0,0.0,0.0,0.561,311599.0,46.87
0,2019-02-01 02:00:00,331710.0,2019.0,2.0,1.0,2.0,5.0,0.0,0.0,0.0,8.80,640.0,0.0,0.0,0.561,311599.0,46.87
0,2019-02-01 03:00:00,331710.0,2019.0,2.0,1.0,3.0,5.0,0.0,0.0,0.0,8.30,660.0,0.0,0.0,0.561,311599.0,46.87
0,2019-02-01 04:00:00,331710.0,2019.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,9.70,550.0,0.0,0.0,0.561,311599.0,46.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2020-01-31 19:00:00,NaN,2020.0,1.0,31.0,19.0,5.0,0.0,0.0,0.0,16.77,591.0,0.0,0.0,0.536,339021.0,43.13
2756,2020-01-31 20:00:00,NaN,2020.0,1.0,31.0,20.0,5.0,0.0,0.0,0.0,16.13,594.0,0.0,0.0,0.536,339021.0,43.13
2756,2020-01-31 21:00:00,NaN,2020.0,1.0,31.0,21.0,5.0,0.0,0.0,0.0,15.57,612.0,0.0,0.0,0.536,339021.0,43.13


Y guardamos un archivo intermedio:

In [13]:
df.to_csv('data/Modelar_Procesado_conNaN.csv')

Una vez hecho esto, vamos a imputar los NaN. Antes, escalamos los valores de las columnas que hemos añadido con un MinMaxScaler (aunque en este paso no sea relevante): 

In [14]:
df = pd.read_csv('data/Modelar_Procesado_conNaN.csv', index_col='ID')
df['SAMPLETIME'] = pd.to_datetime(df.SAMPLETIME)
df = df.sort_values(["ID", 'SAMPLETIME'])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Primero vamos a escalar los nuevos valores

In [15]:
scaler = StandardScaler()
df[['Temp', 'H.Rel', 'R.Sol', 'Precip', 'OCUPACIÓN DIARIA COMPLETA', 'PERNOTACIONES TOTAL', 'OCUPACIÓN TOTAL']] = scaler.fit_transform(df[['Temp', 'H.Rel', 'R.Sol', 'Precip', 'OCUPACIÓN DIARIA COMPLETA', 'PERNOTACIONES TOTAL', 'OCUPACIÓN TOTAL']])

In [16]:
df

,SAMPLETIME,READINGINTEGER,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL
ID,,,,,,,,,,,,,,,,,
0,2019-02-01 00:00:00,331710.0,2019.0,2.0,1.0,0.0,5.0,0.0,0.0,0.0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
0,2019-02-01 01:00:00,331710.0,2019.0,2.0,1.0,1.0,5.0,0.0,0.0,0.0,-1.551055,0.248867,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
0,2019-02-01 02:00:00,331710.0,2019.0,2.0,1.0,2.0,5.0,0.0,0.0,0.0,-1.566477,0.072515,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
0,2019-02-01 03:00:00,331710.0,2019.0,2.0,1.0,3.0,5.0,0.0,0.0,0.0,-1.643589,0.190083,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
0,2019-02-01 04:00:00,331710.0,2019.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,-1.427676,-0.456541,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2020-01-31 19:00:00,NaN,2020.0,1.0,31.0,19.0,5.0,0.0,0.0,0.0,-0.337318,-0.215526,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853
2756,2020-01-31 20:00:00,NaN,2020.0,1.0,31.0,20.0,5.0,0.0,0.0,0.0,-0.436021,-0.197891,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853
2756,2020-01-31 21:00:00,NaN,2020.0,1.0,31.0,21.0,5.0,0.0,0.0,0.0,-0.522386,-0.092080,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853


Y después aplicamos el `IterativeImputer` a cada índice una vez que hemos realizado el `.interpolate()`:

In [17]:
imputer=IterativeImputer(max_iter=10, random_state=42)

# Para cada índice
for i in tqdm(list_i):
    df_temp = df.loc[i]
    
    imputed_dataset = df.loc[i].drop(['SAMPLETIME'],axis=1).interpolate()
    
    imputed_dataset = imputer.fit_transform(imputed_dataset)
    
    df.loc[i, 'READINGINTEGER'] = pd.DataFrame(imputed_dataset).iloc[:,0].values

100%|██████████| 775/775 [03:08<00:00,  4.12it/s]


#### Correción de valores decrecientes y "escalones"

El problema, como hemos visto en el script de exploración, es que se nos vuelven a qudar valores decrecientes de `READINGINTEGER` y saltos entre horas. Los corregimos igual que en la ocasión anterior:

In [18]:
# Para cada contador (índice)
for i in tqdm(list_i):
    df_temp = df.loc[i]
        
    # Comprobamos uno por uno todos los valores comparándolos con el anterior
    for j in range(1,len(df_temp)):
        if df_temp.iloc[j]["READINGINTEGER"] < df_temp.iloc[j-1]["READINGINTEGER"]:
            
            new = df_temp.iloc[j-1]["READINGINTEGER"] - df_temp.iloc[j]["READINGINTEGER"]
            
            df.loc[i].loc[df_temp['SAMPLETIME'] >= df_temp.iloc[j]["SAMPLETIME"],'READINGINTEGER'] = df_temp.loc[df_temp['SAMPLETIME'] >= df_temp.iloc[j]["SAMPLETIME"],'READINGINTEGER'] + new

  0%|          | 1/775 [00:01<15:36,  1.21s/it]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
100%|██████████| 775/775 [27:42<00:00,  2.15s/it]


In [19]:
pd.set_option('mode.chained_assignment', None)

plt.rcParams["figure.figsize"] = (15,8)

MENOS_FILTRO = [1136]

# Para cada contador (índice)
for i in tqdm(list_i):
    df_temp = df.loc[i]
            
    df_temp['DELTA_I'] = df_temp.diff()['READINGINTEGER']
    df_temp['DELTA_H'] = df_temp.diff()['SAMPLETIME']
            
    diff = df_temp.READINGINTEGER.max() - df_temp.READINGINTEGER.min()

    maxDelta = diff / 200  # Unos 200 para 8759 valores
        
    if i in MENOS_FILTRO:
        maxDelta = maxDelta * 2
            
    # Filtramos si cambia de una hora para otra más de lo que "debería"
    df_temp_2 = df_temp.loc[(df_temp['DELTA_I'] > maxDelta) & (df_temp['DELTA_H'] == '0 days 01:00:00')]
            
    if (df_temp_2.count()["SAMPLETIME"] != 0) and (maxDelta > 200):
     
        for j in range(0,len(df_temp_2)):
            filtro = df_temp['SAMPLETIME'] >= df_temp_2.iloc[j]["SAMPLETIME"]

            # A todos los valores posteriores al salto, les restamos la diferencia de READINGINTEGER entre j y j-1 (que siempre > 0)
            df_temp.loc[filtro,'READINGINTEGER'] = df_temp.loc[filtro,'READINGINTEGER'] - df_temp_2.iloc[j]["DELTA_I"]
                            
        df.loc[i] = df_temp

plt.rcParams["figure.figsize"] = (15,6)

100%|██████████| 775/775 [10:37<00:00,  1.21it/s]


#### Creamos la variable `DELTA` definitiva

Por último, una vez que tenemos nuestro `READINGINTEGER` corregido, podemos crear una variable `DELTA` propia, que será la que queremos predecir

In [20]:
for i in tqdm(df.index.drop_duplicates()):
    df.loc[i, 'DELTA'] = df.loc[i].diff()['READINGINTEGER']

100%|██████████| 2747/2747 [06:13<00:00,  7.35it/s]


Los primeros valores se nos quedan con `DELTA` `NaN`, los reemplazamos por 0

In [21]:
df['DELTA'] = df['DELTA'].fillna(0)

In [22]:
df

,SAMPLETIME,READINGINTEGER,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL,DELTA
ID,,,,,,,,,,,,,,,,,,
0,2019-02-01 00:00:00,331710.000000,2019.0,2.0,1.0,0.0,5.0,0.0,0.0,0.0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019-02-01 01:00:00,331710.000000,2019.0,2.0,1.0,1.0,5.0,0.0,0.0,0.0,-1.551055,0.248867,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019-02-01 02:00:00,331710.000000,2019.0,2.0,1.0,2.0,5.0,0.0,0.0,0.0,-1.566477,0.072515,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019-02-01 03:00:00,331710.000000,2019.0,2.0,1.0,3.0,5.0,0.0,0.0,0.0,-1.643589,0.190083,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019-02-01 04:00:00,331710.000000,2019.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,-1.427676,-0.456541,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2020-01-31 19:00:00,349782.850208,2020.0,1.0,31.0,19.0,5.0,0.0,0.0,0.0,-0.337318,-0.215526,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,0.0
2756,2020-01-31 20:00:00,349782.850208,2020.0,1.0,31.0,20.0,5.0,0.0,0.0,0.0,-0.436021,-0.197891,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,0.0
2756,2020-01-31 21:00:00,349782.850208,2020.0,1.0,31.0,21.0,5.0,0.0,0.0,0.0,-0.522386,-0.092080,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,0.0


Y guardamos el DataFrame final

In [23]:
df.to_csv('data/Modelar_UH2022_DEF.csv')

## Crear DataFrame de test

Ahora, vamos a crear el DataFrame de test que usaremos para predecir:

In [24]:
df = pd.read_csv('data/Modelar_Procesado_conNaN.csv', index_col='ID')
df['SAMPLETIME'] = pd.to_datetime(df.SAMPLETIME)
df = df.sort_values(["ID", 'SAMPLETIME'])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [25]:
dic = {'ID': [], 'SAMPLETIME': []}

for id in df.index.drop_duplicates():
    for dia in range(1,15):
        for hora in range(24):
            fecha = "2019-02-" + str(dia) + " " + str(hora) + ":00:00"
            dic['ID'].append(id)
            dic['SAMPLETIME'].append(fecha)
            
df_test = pd.DataFrame(dic)

df_test['SAMPLETIME'] = pd.to_datetime(df_test.SAMPLETIME)
df_test = df_test.astype({"ID": int})
df_test = df_test.set_index("ID")
df_test = df_test.sort_values(["ID", 'SAMPLETIME'])

In [26]:
df_test

,SAMPLETIME
ID,
0,2019-02-01 00:00:00
0,2019-02-01 01:00:00
0,2019-02-01 02:00:00
0,2019-02-01 03:00:00
0,2019-02-01 04:00:00
...,...
2756,2019-02-14 19:00:00
2756,2019-02-14 20:00:00
2756,2019-02-14 21:00:00


Tenemos que añadirle las columnas que también añadimos al del entrenamiento

In [27]:
df_test['YEAR'] = df_test['SAMPLETIME'].dt.strftime('%Y').astype(int)
df_test['MONTH'] = df_test['SAMPLETIME'].dt.strftime('%m').astype(int)
df_test['DAY'] = df_test['SAMPLETIME'].dt.strftime('%e').astype(int)
df_test['HOUR'] = df_test['SAMPLETIME'].dt.strftime('%H').astype(int)
df_test['DAYWEEK'] = df_test['SAMPLETIME'].dt.strftime('%u').astype(int)

In [28]:
#FESTIVIDAD NACIONAL
anos = [2019,2020]

count_n = [[1,1,1,1,1,1,2],[2,1,1,1,1,2]]
meses_n = [[1,4,5,8,10,11,12],[1,4,5,8,10,12]]
nacionales = [[1,19,1,15,12,1,6,25],[1,6,10,1,15,12,8,25]]

count_a = [[2,1,1],[1,2,1,1]]
meses_a = [[4,6,10],[3,4,6,10]]
autonomicos = [[22,25,24,9],[19,13,25,24,9]]

count_l = [[1],[1]]
meses_l = [[1],[1]]
locales = [[22],[22]]

df_test['NACIONAL'] = 0
df_test['AUTONOMICO'] = 0
df_test['LOCAL'] = 0

nombres = ["NACIONAL", "AUTONOMICO", "LOCAL"]
aux_c = [count_n, count_a, count_l]
aux_m = [meses_n, meses_a, meses_l]
aux_d = [nacionales, autonomicos, locales]

for colum in nombres:
    meses = aux_m.pop(0)
    count = aux_c.pop(0)
    days = aux_d.pop(0)
    for ano in anos:
        meses2 = meses.pop(0)
        count2 = count.pop(0)
        days2 = days.pop(0)
        for m in meses2:
            cuenta = count2.pop(0)
            for d in range(0, cuenta): 
                df_test.loc[(df_test["YEAR"]==ano) & (df_test["MONTH"]==m) & (df_test["DAY"]==days2.pop(0)), colum] = 1
df_test


,SAMPLETIME,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL
ID,,,,,,,,,
0,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0
0,2019-02-01 01:00:00,2019,2,1,1,5,0,0,0
0,2019-02-01 02:00:00,2019,2,1,2,5,0,0,0
0,2019-02-01 03:00:00,2019,2,1,3,5,0,0,0
0,2019-02-01 04:00:00,2019,2,1,4,5,0,0,0
...,...,...,...,...,...,...,...,...,...
2756,2019-02-14 19:00:00,2019,2,14,19,4,0,0,0
2756,2019-02-14 20:00:00,2019,2,14,20,4,0,0,0
2756,2019-02-14 21:00:00,2019,2,14,21,4,0,0,0


In [29]:
df2 = pd.read_csv('data/PrecHumTempValencia.csv', sep='\;', encoding='ISO-8859-1')
df2 = df2.replace('"','', regex=True)
df2 = df2.replace(',','.', regex=True)
df2 = df2.rename(columns={"\"Fecha Hora": "Fecha", "Precip\"": "Precip"})
df2["Temp"] = pd.to_numeric(df2["Temp"])
df2["H.Rel"] = pd.to_numeric(df2["H.Rel"])
df2["R.Sol"] = pd.to_numeric(df2["R.Sol"])
df2["Precip"] = pd.to_numeric(df2["Precip"])
df2['Fecha'] = pd.to_datetime(df2.Fecha)
df2 = df2.set_index("Fecha")
df2

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Estación,Temp,H.Rel,R.Sol,Precip
Fecha,,,,,
2020-01-01 00:00:00,Avda. Francia,6.6,890.0,0.0,0.0
2020-01-01 01:00:00,Avda. Francia,6.2,890.0,0.0,0.0
2020-01-01 02:00:00,Avda. Francia,5.9,900.0,0.0,0.0
2020-01-01 03:00:00,Avda. Francia,5.5,900.0,0.0,0.0
2020-01-01 04:00:00,Avda. Francia,5.3,890.0,0.0,0.0
...,...,...,...,...,...
2018-12-23 01:00:00,Politécnico,11.5,760.0,20.0,0.0
2018-12-23 02:00:00,Politécnico,11.0,770.0,10.0,0.0
2018-12-23 03:00:00,Politécnico,10.9,770.0,10.0,0.0


In [30]:
df2 = df2.sort_index()
df_group = df2.groupby("Fecha").mean()
df_group

,Temp,H.Rel,R.Sol,Precip
Fecha,,,,
2018-01-01 00:00:00,14.20,380.0,0.0,0.0
2018-01-01 01:00:00,12.80,460.0,0.0,0.0
2018-01-01 02:00:00,12.10,460.0,0.0,0.0
2018-01-01 03:00:00,11.20,450.0,0.0,0.0
2018-01-01 04:00:00,10.60,460.0,0.0,0.0
...,...,...,...,...
2020-12-31 19:00:00,11.15,442.0,4.0,0.0
2020-12-31 20:00:00,10.51,474.0,14.0,0.0
2020-12-31 21:00:00,10.37,506.0,4.0,0.0


In [31]:
imp = SimpleImputer(missing_values=np.nan)
idf = pd.DataFrame(imp.fit_transform(df_group))
idf.columns = df_group.columns
idf.index = df_group.index
idf

,Temp,H.Rel,R.Sol,Precip
Fecha,,,,
2018-01-01 00:00:00,14.20,380.0,0.0,0.0
2018-01-01 01:00:00,12.80,460.0,0.0,0.0
2018-01-01 02:00:00,12.10,460.0,0.0,0.0
2018-01-01 03:00:00,11.20,450.0,0.0,0.0
2018-01-01 04:00:00,10.60,460.0,0.0,0.0
...,...,...,...,...
2020-12-31 19:00:00,11.15,442.0,4.0,0.0
2020-12-31 20:00:00,10.51,474.0,14.0,0.0
2020-12-31 21:00:00,10.37,506.0,4.0,0.0


In [32]:
dfjoin = df_test.set_index('SAMPLETIME', append=True).reset_index(level=0)
dfjoin = dfjoin.join(idf)
dfjoin = dfjoin.set_index('ID', append=True).reset_index(level=0)
dfjoin = dfjoin.rename(columns={"level_0": "SAMPLETIME"})
dfjoin

,SAMPLETIME,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip
ID,,,,,,,,,,,,,
0,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0
1,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0
2,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0
3,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0
4,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0
2747,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0
2748,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0


In [33]:
dfjoin['FECHA'] = dfjoin['SAMPLETIME'].dt.strftime('%F')
dfjoin['FECHA'] = pd.to_datetime(dfjoin.FECHA)
precios_df = pd.read_excel('data\DATOS DIAS.xlsx')
precios_df = precios_df.astype({'FECHA': 'datetime64[ns]'})

precios_df = precios_df.set_index('FECHA')
df3 = dfjoin.set_index('FECHA', append=True).reset_index(level=0)
df3 = df3.join(precios_df)
df3 = df3.set_index('ID', append=True).reset_index(level=0)
dfjoin = df3.rename(columns={"level_0": "FECHA"})
dfjoin

,FECHA,SAMPLETIME,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,TARIFA DIARIA COMPLETA,OCUPACIÓN DIARIA COMPLETA,TARIFA DIARIA COMPARTIDA,OCUPACIÓN DIARIA COMPARTIDA,TARIFA DIARIA PRIVADA,OCUPACIÓN DIARIA PRIVADA
ID,,,,,,,,,,,,,,,,,,,,
0,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
1,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
2,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
3,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
4,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,71.92,0.561,41.33,0.059,28.45,0.378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2019-02-14,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0,65.42,0.545,41.33,0.059,26.76,0.388
2747,2019-02-14,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0,65.42,0.545,41.33,0.059,26.76,0.388
2748,2019-02-14,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0,65.42,0.545,41.33,0.059,26.76,0.388


In [34]:
dfjoin['FECHA2'] = dfjoin['SAMPLETIME'].dt.strftime('%Y-%m')
dfjoin['FECHA2'] = pd.to_datetime(dfjoin.FECHA2)
hoteles_df = pd.read_excel('data\VIAJEROS Y PERNOTACIONES DE HOTELES Y APARTAMENTOS.xlsx')
hoteles_df = hoteles_df.drop(hoteles_df.columns[[0,1,2,3,4,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25]], axis=1)

hoteles_df['AÑO'] = hoteles_df['AÑO'].astype(str)
hoteles_df['MES'] = hoteles_df['MES'].astype(str)
hoteles_df['FECHA2'] = hoteles_df['AÑO'] + "-" + hoteles_df['MES']
hoteles_df['FECHA2'] = pd.to_datetime(hoteles_df.FECHA2)
hoteles_df = hoteles_df.set_index('FECHA2')

df3 = dfjoin.set_index('FECHA2', append=True).reset_index(level=0)
df3 = df3.join(hoteles_df)
df3 = df3.set_index('ID', append=True).reset_index(level=0)
dfjoin = df3.rename(columns={"level_0": "FECHA2"})
dfjoin

,FECHA2,FECHA,SAMPLETIME,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,...,TARIFA DIARIA COMPLETA,OCUPACIÓN DIARIA COMPLETA,TARIFA DIARIA COMPARTIDA,OCUPACIÓN DIARIA COMPARTIDA,TARIFA DIARIA PRIVADA,OCUPACIÓN DIARIA PRIVADA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL,MES,AÑO
ID,,,,,,,,,,,,,,,,,,,,,
0,2019-02-01,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
1,2019-02-01,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
2,2019-02-01,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
3,2019-02-01,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
4,2019-02-01,2019-02-01,2019-02-01 00:00:00,2019,2,1,0,5,0,0,...,71.92,0.561,41.33,0.059,28.45,0.378,311599,46.87,2,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2019-02-01,2019-02-14,2019-02-14 23:00:00,2019,2,14,23,4,0,0,...,65.42,0.545,41.33,0.059,26.76,0.388,311599,46.87,2,2019
2747,2019-02-01,2019-02-14,2019-02-14 23:00:00,2019,2,14,23,4,0,0,...,65.42,0.545,41.33,0.059,26.76,0.388,311599,46.87,2,2019
2748,2019-02-01,2019-02-14,2019-02-14 23:00:00,2019,2,14,23,4,0,0,...,65.42,0.545,41.33,0.059,26.76,0.388,311599,46.87,2,2019


In [35]:
dfjoin = dfjoin.drop(["FECHA2", "FECHA", "MES", "AÑO"], axis=1)

In [36]:
dfjoin = dfjoin.drop(["TARIFA DIARIA COMPLETA", "TARIFA DIARIA COMPARTIDA", "OCUPACIÓN DIARIA COMPARTIDA", "TARIFA DIARIA PRIVADA", "OCUPACIÓN DIARIA PRIVADA"], axis=1)

### Scaler

Aplicamos el ``StandardScaler`` que creamos anteriormente

In [44]:
dfjoin

,SAMPLETIME,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL
ID,,,,,,,,,,,,,,,,
0,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,0.561,311599,46.87
1,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,0.561,311599,46.87
2,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,0.561,311599,46.87
3,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,0.561,311599,46.87
4,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,9.1,700.0,0.0,0.0,0.561,311599,46.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0,0.545,311599,46.87
2747,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0,0.545,311599,46.87
2748,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,9.3,780.0,10.0,0.0,0.545,311599,46.87


In [45]:
dfjoin[['Temp', 'H.Rel', 'R.Sol', 'Precip', 'OCUPACIÓN DIARIA COMPLETA', 'PERNOTACIONES TOTAL', 'OCUPACIÓN TOTAL']] = scaler.transform(dfjoin[['Temp', 'H.Rel', 'R.Sol', 'Precip', 'OCUPACIÓN DIARIA COMPLETA', 'PERNOTACIONES TOTAL', 'OCUPACIÓN TOTAL']])

In [46]:
dfjoin

,SAMPLETIME,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL
ID,,,,,,,,,,,,,,,,
0,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
1,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
2,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
3,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
4,2019-02-01 00:00:00,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,-1.489365,0.895491,-0.704257,-0.100731,-0.690628,-1.656154,-1.228948
2747,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,-1.489365,0.895491,-0.704257,-0.100731,-0.690628,-1.656154,-1.228948
2748,2019-02-14 23:00:00,2019,2,14,23,4,0,0,0,-1.489365,0.895491,-0.704257,-0.100731,-0.690628,-1.656154,-1.228948


Y una vez que lo tenemos, guardamos el resultado:

In [47]:
dfjoin.to_csv('data/Predecir_DEF.csv')

## Predicción

In [164]:
df_test = pd.read_csv('data/Predecir_DEF.csv')
df_test = df_test.drop(['SAMPLETIME'],axis=1)
df_test

,ID,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL
0,0,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
1,1,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
2,2,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
3,3,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
4,4,2019,2,1,0,5,0,0,0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922987,2746,2019,2,14,23,4,0,0,0,-1.489365,0.895491,-0.704257,-0.100731,-0.690628,-1.656154,-1.228948
922988,2747,2019,2,14,23,4,0,0,0,-1.489365,0.895491,-0.704257,-0.100731,-0.690628,-1.656154,-1.228948
922989,2748,2019,2,14,23,4,0,0,0,-1.489365,0.895491,-0.704257,-0.100731,-0.690628,-1.656154,-1.228948
922990,2749,2019,2,14,23,4,0,0,0,-1.489365,0.895491,-0.704257,-0.100731,-0.690628,-1.656154,-1.228948


### Entorno de pruebas

Para realizar las nuevas predicciones, vamos a crear un mejor entorno de pruebas para poder realizar una validación sobre los distintos modelos.

In [165]:
df = pd.read_csv('data/Modelar_UH2022_DEF.csv', index_col='ID', parse_dates=True)
df = df.reset_index(drop=False)
df

,ID,SAMPLETIME,READINGINTEGER,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL,DELTA
0,0,2019-02-01 00:00:00,331710.000000,2019.0,2.0,1.0,0.0,5.0,0.0,0.0,0.0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
1,0,2019-02-01 01:00:00,331710.000000,2019.0,2.0,1.0,1.0,5.0,0.0,0.0,0.0,-1.551055,0.248867,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
2,0,2019-02-01 02:00:00,331710.000000,2019.0,2.0,1.0,2.0,5.0,0.0,0.0,0.0,-1.566477,0.072515,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
3,0,2019-02-01 03:00:00,331710.000000,2019.0,2.0,1.0,3.0,5.0,0.0,0.0,0.0,-1.643589,0.190083,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
4,0,2019-02-01 04:00:00,331710.000000,2019.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,-1.427676,-0.456541,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24060968,2756,2020-01-31 19:00:00,349782.850208,2020.0,1.0,31.0,19.0,5.0,0.0,0.0,0.0,-0.337318,-0.215526,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,0.0
24060969,2756,2020-01-31 20:00:00,349782.850208,2020.0,1.0,31.0,20.0,5.0,0.0,0.0,0.0,-0.436021,-0.197891,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,0.0
24060970,2756,2020-01-31 21:00:00,349782.850208,2020.0,1.0,31.0,21.0,5.0,0.0,0.0,0.0,-0.522386,-0.092080,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,0.0
24060971,2756,2020-01-31 22:00:00,349782.850208,2020.0,1.0,31.0,22.0,5.0,0.0,0.0,0.0,-0.574822,-0.015661,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,0.0


Así, vamos a establecer como datos de validación las semanas del 18 al 24 y del 25 al 31 de enero, de una forma anláloga a lo que se pide en el reto

- Semana 1: 18, 19, 20, 21, 22, 23, 24
- Semana 2: 25, 26, 27, 28, 29, 30, 31


In [166]:
X, y = df.iloc[:,:-1], df.iloc[:,-1]

df_train = df.loc[df['SAMPLETIME'] < '2020-01-18']
df_test = df.loc[df['SAMPLETIME'] >= '2020-01-18']

X_train = X.loc[df['SAMPLETIME'] < '2020-01-18']
X_test = X.loc[df['SAMPLETIME'] >= '2020-01-18']

X_train = X_train.drop(['SAMPLETIME','READINGINTEGER'],axis=1)
X_test = X_test.drop(['SAMPLETIME','READINGINTEGER'],axis=1)

y_train = y.loc[df['SAMPLETIME'] < '2020-01-18']
y_test = y.loc[df['SAMPLETIME'] >= '2020-01-18']

### RMSE

Y definimos la función a la que, pasándole una predicción, calcule el RMSE de la misma en el formato de la competición:

In [71]:
def calculate_RMSE(preds):
    X_test_preds = X_test.set_index(['ID'])
    X_test_preds['PRED'] = preds
    X_test_preds = X_test_preds.groupby(['DAY','ID']).sum()
    
    # Creamos el DataFrame tipo salida final para los datos predichos
    nuevodf = pd.DataFrame()

    for i in range (18,25):
        nuevodf[str(i)] = X_test_preds.loc[i, 'PRED']
        
    nuevodf["sem1"] = nuevodf["18"] + nuevodf["19"] + nuevodf["20"] + nuevodf["21"] + nuevodf["22"] + nuevodf["23"] + nuevodf["24"]
    nuevodf["sem2"] = X_test_preds.loc[25, 'PRED'] + X_test_preds.loc[26, 'PRED'] + X_test_preds.loc[27, 'PRED'] + X_test_preds.loc[28, 'PRED'] + X_test_preds.loc[29, 'PRED'] + X_test_preds.loc[30, 'PRED'] + X_test_preds.loc[31, 'PRED']

    X_test_origin = df_test.groupby(['DAY','ID']).sum()
    
    # Creamos el DataFrame tipo salida final para los datos originales
    nuevodf2 = pd.DataFrame()

    for i in range (18,25):
        nuevodf2[str(i)] = X_test_origin.loc[i, 'DELTA']

    nuevodf2["sem1"] = nuevodf2["18"] + nuevodf2["19"] + nuevodf2["20"] + nuevodf2["21"] + nuevodf2["22"] + nuevodf2["23"] + nuevodf2["24"]
    nuevodf2["sem2"] = X_test_origin.loc[25, 'DELTA'] + X_test_origin.loc[26, 'DELTA'] + X_test_origin.loc[27, 'DELTA'] + X_test_origin.loc[28, 'DELTA'] + X_test_origin.loc[29, 'DELTA'] + X_test_origin.loc[30, 'DELTA'] + X_test_origin.loc[31, 'DELTA']

    # Una vez los tenemos, podemos calcular la métrica
    rmse1 = 0
    for i in range(7):
        diffs = np.square(nuevodf.iloc[:,i] - nuevodf2.iloc[:,i])
        rmse1 = rmse1 + np.sqrt(np.sum(diffs))
        
    rmse1 = rmse1 / 7
    print("RMSE 1: ", rmse1)

    rmse2 = 0
    for i in range(2):
        diffs = np.square(nuevodf.iloc[:,i+7] - nuevodf2.iloc[:,i+7])
        rmse2 = rmse2 + np.sqrt(np.sum(diffs))
        
    rmse2 = rmse2 / 2
    print("RMSE 2: ", rmse2)

    print("RMSE TOTAL: ", (rmse1 + rmse2)/2)
    
    return ((rmse1 + rmse2)/2)

## Búsqueda de modelos

Un problema que nos vamos a encontrar con respecto a la búsqueda de modelos, es que debido al gran tiempo de ejecución (hay que tener en cuenta que contamos con millones de filas para entrenar), va a resultar muy dificil realizar un proceso de `Grid Search` para optimizar los hiperparámetros de los modelos. Así, puede resultar preferible probar varios tipos de algoritmos con hiperparámetros típicos, para posteriormente realizar un "ensemble" con los que mejor resultado obtengan mejorando así la predicción que realizaría cualquiera de ellos por separado.

### `XGBRegressor` (RMSE 496038473 entrega intermedia)

Vamos a empezar ejecutando un `XGBRegressor` para obtener una métrica comparable a la que enviamos a la entrega intermedia:

In [52]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

calculate_RMSE(preds)

RMSE 1:  114520.81021776152
RMSE 2:  668251.5953669534
RMSE TOTAL:  391386.2027923575


391386.2027923575

### Array de 0's  (RMSE 496038479 entrega intermedia)

Del mismo modo, también vamos a calcular la nueva métrica para una predicción de consumo 0 para todos los contadores:

In [6]:
preds0 = np.zeros(len(X_test))
calculate_RMSE(preds0)

RMSE 1:  116919.28435833617
RMSE 2:  687642.7560923578
RMSE TOTAL:  402281.02022534696


402281.02022534696

### Un `XGBRegressor` para cada ID

Como vemos que el problema del `XGBRegressor` es que no realiza una predicción distinta para cada ID, podemos obligarlo a ello entrenando un  modelo para cada ID. El problema de esto es que no se podrá enriquecer con los datos de los demás contadores para predecir.

In [7]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 100)

predicciones = np.array([])

for i in tqdm(df_train['ID'].drop_duplicates()):
    filtro = X_train['ID'] == i
    
    X_train_2, y_train_2 = X_train[filtro], y_train[filtro]

    X_test_2 = X_test[X_test['ID'] == i]
    
    xg_reg.fit(X_train_2, y_train_2)

    preds_temp = xg_reg.predict(X_test_2)
    
    predicciones = np.concatenate((predicciones, preds_temp), axis=None)

calculate_RMSE(predicciones)

100%|██████████| 2747/2747 [15:08<00:00,  3.02it/s]


RMSE 1:  111206.71878926022
RMSE 2:  542315.6303553774
RMSE TOTAL:  326761.17457231885


326761.17457231885

Aún así, vemos como finalmente mejoramos al modelo previo

### `IterativeImputer` para cada ID

Anteriormente, vimos como el `IterativeImputer` funcionaba mejor que los modelos predictivos para extrapolar los datos. Así, podemos probar a utilizarlo para realizar nuestras predicciones, ya que al final acaban siendo una extrapolación de 14 días. Primero, vamos a realizarlo igual que anteriormente para cada ID:

In [8]:
df_test2 = df_test.copy()
df_test2["DELTA"] = np.nan
df2 = df_train.append(df_test2)
df2 = df2.drop(['SAMPLETIME','READINGINTEGER'],axis=1)
df2 = df2.set_index("ID")
df2

,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL,DELTA
ID,,,,,,,,,,,,,,,,
0,2019.0,2.0,1.0,0.0,5.0,0.0,0.0,0.0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019.0,2.0,1.0,1.0,5.0,0.0,0.0,0.0,-1.551055,0.248867,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019.0,2.0,1.0,2.0,5.0,0.0,0.0,0.0,-1.566477,0.072515,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019.0,2.0,1.0,3.0,5.0,0.0,0.0,0.0,-1.643589,0.190083,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
0,2019.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,-1.427676,-0.456541,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,2020.0,1.0,31.0,19.0,5.0,0.0,0.0,0.0,-0.337318,-0.215526,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,NaN
2756,2020.0,1.0,31.0,20.0,5.0,0.0,0.0,0.0,-0.436021,-0.197891,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,NaN
2756,2020.0,1.0,31.0,21.0,5.0,0.0,0.0,0.0,-0.522386,-0.092080,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853,NaN


In [9]:
imputer = IterativeImputer(max_iter=10, random_state=42)

# Para cada índice
for i in tqdm(df2.index.drop_duplicates()):
    imputed_dataset = imputer.fit_transform(df2.loc[i])
    
    df2.loc[i, 'DELTA'] = pd.DataFrame(imputed_dataset).iloc[:,-1].values
    
df_temp = df2.loc[(df2['MONTH'] == 1) & (df2['YEAR'] == 2020) & (df2["DAY"] >= 18)]
preds25 = df_temp["DELTA"]
preds25[preds25 < 0] = 0

calculate_RMSE(preds25)

100%|██████████| 2747/2747 [13:43<00:00,  3.33it/s]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


RMSE 1:  86819.84864578281
RMSE 2:  402595.5544100449
RMSE TOTAL:  244707.70152791386


244707.70152791386

Y obtenemos un resultado mejor que los que teníamos anteriormente

### `GradientBoostingRegressor` para todas las ID

Por último vamos a probar el algoritmo `GradientBoostingRegressor`:

In [24]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(X_train,y_train)
preds_gbID = gb_reg.predict(X_test)

In [25]:
calculate_RMSE(preds_gbID)

RMSE 1:  98210.930963545
RMSE 2:  524341.2581244389
RMSE TOTAL:  311276.09454399196


311276.09454399196

### `GradientBoostingRegressor` para cada ID

Y lo volvemos a ejecutar creando un modelo para cada ID, que como podemos ver obtiene un mejor resultado:

In [10]:
gb_reg = GradientBoostingRegressor()

preds_gb = np.array([])

for i in tqdm(df_train['ID'].drop_duplicates()):
    filtro = X_train['ID'] == i
    
    X_train_2, y_train_2 = X_train[filtro], y_train[filtro]

    X_test_2 = X_test[X_test['ID'] == i]
    
    gb_reg.fit(X_train_2, y_train_2)

    preds_temp = gb_reg.predict(X_test_2)
    
    preds_gb = np.concatenate((preds_gb, preds_temp), axis=None)

calculate_RMSE(preds_gb)

100%|██████████| 2747/2747 [35:44<00:00,  1.28it/s]


RMSE 1:  96135.5103913295
RMSE 2:  443408.03769107885
RMSE TOTAL:  269771.7740412042


269771.7740412042

### Ensemble (RMSE 496038384,95 entrega fase local)

Vamos a probar a hacer un emsemble con los cuatro mejores predictores (`XGBoost`, `IterativeImputer` y `GradientBoostingRegressor` para cada ID, además del `GradientBoostingRegressor` global). Podemos ver cómo conseguimos mejorar nuestras soluciones anteriores al reducir la varianza y lograr un modelo más robusto:

In [111]:
preds_gb = np.load("preds_gb.npy")

In [67]:
calculate_RMSE((predicciones + preds25 + preds_gb + preds_gbID)/4)

RMSE 1:  78422.87830375516
RMSE 2:  342323.6790590262
RMSE TOTAL:  210373.2786813907


210373.2786813907

## Sustitución de los valores de ID por la media

Hemos visto cómo generalmente funciona mejor realizar un modelo para cada contador en lugar de uno general, ya que los modelos en la mayoría de los casos no logran crear valores distintos para cada ID. Por ello, vamos a probar a sustituir dichos ID por la media del `DELTA` de cada contador, para que así si los algoritmos agrupar contadores por sus ID, estos sean similares. Esto lo vamos a conseguir utilizando un `TargetEnconder`:

In [120]:
enc = TargetEncoder(cols=['ID'])
training_set = enc.fit_transform(X_train['ID'], y_train)
test_set = enc.transform(X_test['ID'])

X_train['ID'] = training_set
X_test['ID'] = test_set
X_train

,ID,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL
0,11.704025,2019.0,2.0,1.0,0.0,5.0,0.0,0.0,0.0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
1,11.704025,2019.0,2.0,1.0,1.0,5.0,0.0,0.0,0.0,-1.551055,0.248867,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
2,11.704025,2019.0,2.0,1.0,2.0,5.0,0.0,0.0,0.0,-1.566477,0.072515,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
3,11.704025,2019.0,2.0,1.0,3.0,5.0,0.0,0.0,0.0,-1.643589,0.190083,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
4,11.704025,2019.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,-1.427676,-0.456541,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24060632,0.001529,2020.0,1.0,17.0,19.0,5.0,0.0,0.0,0.0,-0.636511,-1.056137,-0.707636,-0.100731,-1.154762,-1.332439,-1.577853
24060633,0.001529,2020.0,1.0,17.0,20.0,5.0,0.0,0.0,0.0,-0.747552,-0.967961,-0.707636,-0.100731,-1.154762,-1.332439,-1.577853
24060634,0.001529,2020.0,1.0,17.0,21.0,5.0,0.0,0.0,0.0,-0.887895,-0.732825,-0.707636,-0.100731,-1.154762,-1.332439,-1.577853
24060635,0.001529,2020.0,1.0,17.0,22.0,5.0,0.0,0.0,0.0,-0.955753,-0.709312,-0.707636,-0.100731,-1.154762,-1.332439,-1.577853


Una vez hecho esto, vamos a entrenar un `XGBRegressor` con todas las ID:

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 100)
xg_reg.fit(X_train,y_train)
predsX = xg_reg.predict(X_test)

In [119]:
calculate_RMSE(predsX)

RMSE 1:  91885.05031724976
RMSE 2:  475110.8665839761
RMSE TOTAL:  283497.9584506129


283497.9584506129

Y un `GradientBoostingRegressor` de la misma forma:

In [ ]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(X_train,y_train)
preds_gbX = gb_reg.predict(X_test)

In [80]:
calculate_RMSE(preds_gbX)

RMSE 1:  83564.29104346139
RMSE 2:  411243.8100409283
RMSE TOTAL:  247404.05054219486


247404.05054219486

Podemos ver cómo los resultados son suficientemente buenos como para poderlos añadir a nuestro "ensemble" final:

In [169]:
calculate_RMSE((predicciones + preds25 + preds_gb + preds_gbID + preds_gbX + predsX)/6)

RMSE 1:  80330.06059898033
RMSE 2:  351297.3747399936
RMSE TOTAL:  215813.71766948697


215813.71766948697

Aunque el resultado final sea parecido al que teníamos antes, añadir estos dos nuevos modelos nos proporciona una gran robustez frente a nuevos datos desconocidos.

## Entrenamiento del modelo final

Por último, vamos a volver a entrenar los modelos con todos los datos para realizar la predicción final:

Cargamos de nuevo las bases de datos:

In [138]:
df = pd.read_csv('data/Modelar_UH2022_DEF.csv', index_col='ID', parse_dates=True)
df = df.drop(['SAMPLETIME','READINGINTEGER'],axis=1)
df = df.reset_index(drop=False)

In [139]:
df_test = pd.read_csv('data/Predecir_DEF.csv')
df_test = df_test.drop(['SAMPLETIME'],axis=1)

In [140]:
X, y = df.iloc[:,:-1], df.iloc[:,-1]

Entrenamos los distintos modelos:

In [28]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 100)
preds1 = np.array([])

for i in tqdm(X['ID'].drop_duplicates()):
    filtro = X['ID'] == i
    X_2, y_2 = X[filtro], y[filtro]
    X_test_2 = df_test[df_test['ID'] == i]
    xg_reg.fit(X_2, y_2)
    preds_temp = xg_reg.predict(X_test_2)
    preds1 = np.concatenate((preds1, preds_temp), axis=None)

100%|██████████| 2747/2747 [14:30<00:00,  3.15it/s]


In [29]:
df_test2 = df_test.copy()
df_test2["DELTA"] = np.nan
df2 = df.append(df_test2)
df2 = df2.set_index("ID")

imputer = IterativeImputer(max_iter=10, random_state=42)

# Para cada índice
for i in tqdm(df2.index.drop_duplicates()):
    imputed_dataset = imputer.fit_transform(df2.loc[i])
    
    df2.loc[i, 'DELTA'] = pd.DataFrame(imputed_dataset).iloc[:,-1].values
    
df_temp = df2.loc[(df2['MONTH'] == 1) & (df2['YEAR'] == 2020) & (df2["DAY"] >= 18)]
preds2 = df_temp["DELTA"]
preds2[preds2 < 0] = 0

100%|██████████| 2747/2747 [14:32<00:00,  3.15it/s]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [30]:
gb_reg = GradientBoostingRegressor()
preds3 = np.array([])

for i in tqdm(X['ID'].drop_duplicates()):
    filtro = X['ID'] == i
    X_2, y_2 = X[filtro], y[filtro]
    X_test_2 = df_test[df_test['ID'] == i]
    gb_reg.fit(X_2, y_2)
    preds_temp = gb_reg.predict(X_test_2)
    preds3 = np.concatenate((preds3, preds_temp), axis=None)

100%|██████████| 2747/2747 [32:24<00:00,  1.41it/s]


In [31]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(X, y)
preds4 = gb_reg.predict(df_test)

In [39]:
enc = TargetEncoder(cols=['ID'])
training_set = enc.fit_transform(X['ID'], y)
test_set = enc.transform(df_test['ID'])

X['ID'] = training_set
df_test['ID'] = test_set
X

,ID,YEAR,MONTH,DAY,HOUR,DAYWEEK,NACIONAL,AUTONOMICO,LOCAL,Temp,H.Rel,R.Sol,Precip,OCUPACIÓN DIARIA COMPLETA,PERNOTACIONES TOTAL,OCUPACIÓN TOTAL
0,11.78856,2019.0,2.0,1.0,0.0,5.0,0.0,0.0,0.0,-1.520210,0.425219,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
1,11.78856,2019.0,2.0,1.0,1.0,5.0,0.0,0.0,0.0,-1.551055,0.248867,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
2,11.78856,2019.0,2.0,1.0,2.0,5.0,0.0,0.0,0.0,-1.566477,0.072515,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
3,11.78856,2019.0,2.0,1.0,3.0,5.0,0.0,0.0,0.0,-1.643589,0.190083,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
4,11.78856,2019.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,-1.427676,-0.456541,-0.707636,-0.100731,-0.547818,-1.656154,-1.228948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24060968,0.00147,2020.0,1.0,31.0,19.0,5.0,0.0,0.0,0.0,-0.337318,-0.215526,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853
24060969,0.00147,2020.0,1.0,31.0,20.0,5.0,0.0,0.0,0.0,-0.436021,-0.197891,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853
24060970,0.00147,2020.0,1.0,31.0,21.0,5.0,0.0,0.0,0.0,-0.522386,-0.092080,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853
24060971,0.00147,2020.0,1.0,31.0,22.0,5.0,0.0,0.0,0.0,-0.574822,-0.015661,-0.707636,-0.100731,-0.770959,-1.332439,-1.577853


In [40]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.1, max_depth = 5, n_estimators = 100)
xg_reg.fit(X, y)
preds5 = xg_reg.predict(df_test)

In [41]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(X, y)
preds6 = gb_reg.predict(df_test)

Una vez tenemos las 6 predicciones, creamos un ensemble realizando la media de las 6:

In [141]:
preds_final = (preds1 + preds2 + preds3 + preds4 + preds5 + preds6) / 6

Y pasamos las predicciones al formato de `DPNA.txt`

df = pd.read_csv('data/Modelar_UH2022_DEF.csv', index_col='ID', parse_dates=True)
df = df.drop(['SAMPLETIME','READINGINTEGER'],axis=1)
df = df.reset_index(drop=False)
df_test = pd.read_csv('data/Predecir_DEF.csv')
df_test = df_test.drop(['SAMPLETIME'],axis=1)
X, y = df.iloc[:,:-1], df.iloc[:,-1]

In [143]:
df_test_day = df_test.set_index(['ID'])
df_test_day['PRED'] = preds_final.values
df_test_day = df_test_day.groupby(['DAY','ID']).sum()
df_test_day

YEAR  MONTH  HOUR  DAYWEEK  NACIONAL  AUTONOMICO  LOCAL       Temp  \
DAY ID                                                                          
1   0     48456     48   276      120         0           0      0 -25.118793   
    1     48456     48   276      120         0           0      0 -25.118793   
    2     48456     48   276      120         0           0      0 -25.118793   
    3     48456     48   276      120         0           0      0 -25.118793   
    4     48456     48   276      120         0           0      0 -25.118793   
...         ...    ...   ...      ...       ...         ...    ...        ...   
14  2746  48456     48   276       96         0           0      0 -24.686968   
    2747  48456     48   276       96         0           0      0 -24.686968   
    2748  48456     48   276       96         0           0      0 -24.686968   
    2749  48456     48   276       96         0           0      0 -24.686968   
    2756  48456     48   276       96         0           0      0 -24.686968   

             H.Rel     R.Sol    Precip  OCUPACIÓN DIARIA COMPLETA  \
DAY ID                                                              
1   0    -8.899534 -6.266646 -2.417544                 -13.147622   
    1    -8.899534 -6.266646 -2.417544                 -13.147622   
    2    -8.899534 -6.266646 -2.417544                 -13.147622   
    3    -8.899534 -6.266646 -2.417544                 -13.147622   
    4    -8.899534 -6.266646 -2.417544                 -13.147622   
...            ...       ...       ...                        ...   
14  2746  5.561322 -7.624804 -2.417544                 -16.575073   
    2747  5.561322 -7.624804 -2.417544                 -16.575073   
    2748  5.561322 -7.624804 -2.417544                 -16.575073   
    2749  5.561322 -7.624804 -2.417544                 -16.575073   
    2756  5.561322 -7.624804 -2.417544                 -16.575073   

          PERNOTACIONES TOTAL  OCUPACIÓN TOTAL        PRED  
DAY ID                                                      
1   0              -39.747698       -29.494758  233.762612  
    1              -39.747698       -29.494758  144.011454  
    2              -39.747698       -29.494758  160.830499  
    3              -39.747698       -29.494758  313.118131  
    4              -39.747698       -29.494758  242.267939  
...                       ...              ...         ...  
14  2746           -39.747698       -29.494758  433.522382  
    2747           -39.747698       -29.494758  376.015203  
    2748           -39.747698       -29.494758  556.243823  
    2749           -39.747698       -29.494758  334.702905  
    2756           -39.747698       -29.494758  269.478136  

[38458 rows x 15 columns]

In [144]:
nuevodf = pd.DataFrame()

for i in range (1,8):
    nuevodf[str(i)] = df_test_day.loc[i, 'PRED']
    
nuevodf

,1,2,3,4,5,6,7
ID,,,,,,,
0,233.762612,256.409346,317.069874,269.008127,403.707453,287.395769,276.444328
1,144.011454,179.329720,223.122025,194.065832,339.828898,161.820716,185.178769
2,160.830499,140.989938,235.401529,207.508649,318.055906,164.270690,202.336273
3,313.118131,291.327095,331.661378,339.296160,443.434308,306.348321,378.766462
4,242.267939,213.366805,233.811309,294.411501,377.792578,255.110112,308.998903
...,...,...,...,...,...,...,...
2746,219.647741,364.193966,273.499291,398.583596,211.115211,195.457011,232.315891
2747,157.786452,280.719177,290.029406,325.345320,161.608845,130.437306,183.612668
2748,292.955265,383.704274,420.708657,462.915824,310.691901,282.145807,334.484058


In [145]:
nuevodf["sem1"] = nuevodf["1"] + nuevodf["2"] + nuevodf["3"] + nuevodf["4"] + nuevodf["5"] + nuevodf["6"] + nuevodf["7"]

nuevodf["sem2"] = df_test_day.loc[8, 'PRED'] + df_test_day.loc[9, 'PRED'] + df_test_day.loc[10, 'PRED'] + df_test_day.loc[11, 'PRED'] + df_test_day.loc[12, 'PRED'] + df_test_day.loc[13, 'PRED'] + df_test_day.loc[14, 'PRED']
nuevodf

,1,2,3,4,5,6,7,sem1,sem2
ID,,,,,,,,,
0,233.762612,256.409346,317.069874,269.008127,403.707453,287.395769,276.444328,2043.797510,2949.491308
1,144.011454,179.329720,223.122025,194.065832,339.828898,161.820716,185.178769,1427.357415,1925.427930
2,160.830499,140.989938,235.401529,207.508649,318.055906,164.270690,202.336273,1429.393483,2066.724168
3,313.118131,291.327095,331.661378,339.296160,443.434308,306.348321,378.766462,2403.951854,3251.912077
4,242.267939,213.366805,233.811309,294.411501,377.792578,255.110112,308.998903,1925.759145,2607.316106
...,...,...,...,...,...,...,...,...,...
2746,219.647741,364.193966,273.499291,398.583596,211.115211,195.457011,232.315891,1894.812707,2277.303448
2747,157.786452,280.719177,290.029406,325.345320,161.608845,130.437306,183.612668,1529.539175,1744.496712
2748,292.955265,383.704274,420.708657,462.915824,310.691901,282.145807,334.484058,2487.605787,2836.859140


Y guardamos la solución:

In [146]:
nuevodf.to_csv('salida/DPNA.txt', sep='|', float_format='%.2f', header=False)